# Non-Clifford and Error-Corrected Scaling Benchmark

In [ ]:

from benchmarks.runner import BenchmarkRunner
from quasar.simulation_engine import SimulationEngine
from quasar.cost import Backend
from benchmarks.circuits import random_hybrid_circuit, surface_corrected_qaoa_circuit
from benchmarks.memory_utils import max_qubits_statevector
from benchmarks.plot_utils import plot_quasar_vs_baseline_best
from pathlib import Path
import pandas as pd
import json


In [ ]:

# Define circuits and qubit sizes
circuits = {
    'random_hybrid': random_hybrid_circuit,
    'surface_corrected_qaoa': lambda n: surface_corrected_qaoa_circuit(bit_width=n, distance=3, rounds=1),
}
qubit_range = [8, 16, 32, 48, 64]
sv_cap = max_qubits_statevector()
print(f'Statevector cap: {sv_cap} qubits')


In [ ]:

# Benchmark execution
runner = BenchmarkRunner()
engine = SimulationEngine()
records = []
forced_backends = [Backend.STATEVECTOR, Backend.MPS, Backend.DECISION_DIAGRAM]
for name, build in circuits.items():
    for n in qubit_range:
        circ = build(n)
        # Automatic QuASAr
        rec_auto = runner.run_quasar_multiple(circ, engine, repetitions=3, quick=True)
        rec_auto.update({'circuit': name, 'qubits': n, 'framework': 'quasar'})
        records.append(rec_auto)
        # Baselines
        for b in forced_backends:
            if b is Backend.STATEVECTOR and n > sv_cap:
                continue
            rec = runner.run_quasar_multiple(circ, engine, backend=b, repetitions=3, quick=True)
            rec.update({'circuit': name, 'qubits': n, 'framework': b.name})
            records.append(rec)

df = pd.DataFrame(records)
results_dir = Path('../results')
results_dir.mkdir(exist_ok=True, parents=True)
res_path = results_dir / 'non_clifford_scaling_results.json'
df.drop(columns=['result'], errors='ignore').to_json(res_path, orient='records')
df


In [ ]:

# Plot runtime comparison
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(6,4))
plot_quasar_vs_baseline_best(df, annotate_backend=True, ax=ax)
fig_path = Path('../figures/non_clifford_scaling_runtime.png')
fig.savefig(fig_path, dpi=150, bbox_inches='tight')
fig_path


In [ ]:

# Log hardware specs and parameters
meta = {
    'hardware': {
        'cpu': 'Intel i9 13900K',
        'memory_gb': 64,
    },
    'qubit_range': qubit_range,
    'circuits': list(circuits.keys()),
}
meta_path = results_dir / 'non_clifford_scaling_meta.json'
meta_path.write_text(json.dumps(meta, indent=2))
meta
